In [2]:
from twikit import Client
import time
import json
from twikit import TwitterException 

# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

{'flow_token': 'g;171374390967084951:-1713743909726:g55eymvnx5Rrdkubm5GpNhne:14',
 'status': 'success',
 'subtasks': [{'subtask_id': 'LoginSuccessSubtask',
   'open_account': {'user': {'id': 1547081484695216130,
     'id_str': '1547081484695216130',
     'name': 'Eloragh Espie',
     'screen_name': 'EloraghEspie'},
    'next_link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'SuccessExit'},
    'attribution_event': 'login'}},
  {'subtask_id': 'SuccessExit',
   'open_link': {'link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'LoginOpenHomeTimeline'}}},
  {'subtask_id': 'LoginOpenHomeTimeline',
   'open_home_timeline': {'next_link': {'link_type': 'abort',
     'link_id': 'next_link'}}}]}

In [3]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('IGNOREcookies.json')
with open('IGNOREcookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [ ]:
# INPUT: the user handle, a beginning and end of a date range
# OUTPUT: the user handle, the user_ID, and the scraped tweets
def get_all_tweets(handle, since, until):

    # load the cookies so you don't login a million times and get banned
    client.load_cookies('IGNOREcookies.json')
    user_id = client.get_user_by_screen_name(handle)

    # initialize the list we will store our data in
    mass_tweets = []

    # this will pull the first forty tweets
    tweets = client.search_tweet(
        f'from:{handle} since:{since} until:{until}', 'Top', count=20
    )

    # if it returns an empty list, the user had no available tweets during the date time range
    if len(tweets) == 0:
        print('No tweets available')
        return(handle, user_id, [])
    
    # this will keep looking for tweets until a certain number of them has been reached
    else:
        while len(mass_tweets) >= 0 and len(mass_tweets) < 2000:

            # this API provides a 'tweet' object, but we only want the id when we return
            tweets1 = [tweet.id for tweet in tweets]
            mass_tweets += tweets1
            time.sleep(1)  # cooldown so we don't get banned

            # keep pulling tweets until number is hit or there are none left
            tweets = tweets.next()

            # we need to make a check in case we've hit the max number of tweets we can scrape
            # this prevents us from pinging the API for no reason
            if len(tweets) == 0:
                print("No more tweets")
                break
            else:
                print(len(mass_tweets))
                continue

    return(handle, user_id, mass_tweets)

In [ ]:
def process_tweets(handle, name, user_id, tweet_ids):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('IGNOREcookies.json')

    # initialize a list to store all tuples
    tweets = []
    for id in tweet_ids:
        # it throws an Index Error if the tweet has been deleted/ is not available
        try:
            # using the IDs we pulled from above
            tweet = client.get_tweet_by_id(id)
            # get the user id, name, handle, tweet id, tweet text, and date
            tweets.append((int(tweet.id), int(user_id.id), name, handle, str(tweet.text), str(tweet.lang), tweet.is_translatable, tweet.created_at_datetime))
        except IndexError:
            print(f'Index Error: {tweet}')
    
    return tweets

In [ ]:
import time
from math import ceil
import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint

# this is a housekeeping function
# twitter API can throw rate limits
# they're kind of like timeouts
# this function just shows me how much longer I will be in timeout for

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies = client.load_cookies('IGNOREcookies.json')
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [ ]:
# timeout check for scraping tweet IDs
try:
    print(client.search_tweet(
        f'from:JoeBiden since:2020-01-01 until:2021-03-01', 'Latest', count=40
    ))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

In [ ]:
# timeout check for processing tweets
try:
    print(client.get_tweet_by_id(1351951465674276869))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

In [ ]:
# another housekeeping function
# if I'm suddenly getting 403 errors, I can use this to check if I've been banned
# sometimes I just have to go on the browser and reauthenticate

def check_user_status(user_id):
    """
    True if the user is active, otherwise false (not exists or suspended).
    """
    try:
        client.get_user_by_id(user_id)
    except TwitterException as e:
        if str(e).startswith('Invalid user id'):
            return False
        raise e
    else:
        return True

In [ ]:
check_user_status(1547081484695216130)

In [ ]:
# this scrapes the tweet IDs
handle, user_id, tweets = get_all_tweets('realDonaldTrump', since='2014-01-01', until='2024-03-01')
print(handle, user_id, tweets)

In [ ]:
# this cleans them and puts them in the format I need for uploading to the SQLite table
tweets2 = process_tweets(handle, 'Elon Musk', user_id, tweets)

In [ ]:
for tweet in tweets2:
    print(tweet)

In [ ]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE tweets (
          tweet_id INTEGER primary key,
          user_id INTEGER,
          user_name TEXT,
          user_handle TEXT,
          tweet_text TEXT,
          tweet_lang TEXT,
          created_date DATETIME
          )""")

In [ ]:
conn.execute('DROP TABLE tweets')

In [ ]:
# upload data in SQLite table
# storing this data is important since this is an unofficial API
# every time I access it, I am risking not being able to access it again

c.executemany("INSERT INTO tweets VALUES (?,?,?,?,?,?,?)", tweets2)
conn.commit()
